In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [990 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [493 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [960 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [903 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark import SparkFiles

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_data = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yyyy")



In [5]:
# 2. Create a temporary view of the DataFrame.
home_sales_data.createOrReplaceTempView('home_sales')


In [12]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT EXTRACT(year FROM h.date) AS year, ROUND(AVG(price),2) AS Average_Price FROM home_sales AS h
WHERE bedrooms = 4
GROUP BY EXTRACT(year FROM h.date)
SORT BY year
"""
spark.sql(query).show()

+----+-------------+
|year|Average_Price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [13]:
_# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
SELECT date_built, ROUND(AVG(price),2) AS Average_Price FROM home_sales AS h
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
SORT BY date_built
"""
spark.sql(query).show()


+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [14]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
SELECT date_built, ROUND(AVG(price),2) AS Average_Price FROM home_sales AS h
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
SORT BY date_built
"""
spark.sql(query).show()


+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [15]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()

query = """
SELECT ROUND(SUM(view),2) AS Views FROM home_sales
WHERE price >= 350000
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+------+
| Views|
+------+
|358937|
+------+

--- 0.4972555637359619 seconds ---


In [16]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
SELECT ROUND(SUM(view),2) AS Views FROM home_sales
WHERE price >= 350000
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))


+------+
| Views|
+------+
|358937|
+------+

--- 0.3944230079650879 seconds ---


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_data.write.parquet('parquet_delay_basic', mode='overwrite')

In [20]:
# 11. Read the parquet formatted data.
p_home_sales_data=spark.read.parquet('parquet_delay_basic')

In [25]:
# 12. Create a temporary table for the parquet data.
p_home_sales_data.createOrReplaceTempView('p_home_sales')
spark.sql("cache table p_home_sales")


DataFrame[]

In [26]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = """
SELECT ROUND(SUM(view),2) AS Views FROM p_home_sales
WHERE price >= 350000
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+------+
| Views|
+------+
|358937|
+------+

--- 0.32941484451293945 seconds ---


In [27]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")
spark.sql("uncache table p_home_sales")

DataFrame[]

In [30]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False

In [31]:
# 15. Check if the p_home_sales is no longer cached
spark.catalog.isCached("p_home_sales")


False